In [2]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os
from keras.layers import Conv2D, MaxPooling2D,Activation,Flatten, Dense, Dropout,InputLayer
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import face_recognition
from sklearn.model_selection import StratifiedShuffleSplit
from keras.applications.mobilenet import MobileNet, preprocess_input
import keras
from keras.models import Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import cv2
import warnings

In [3]:
path = "./dataset"
name_list = []
ftr_list = []
size = (224, 224)
DIM=224
input_shape = (DIM, DIM, 3)

In [4]:
def createMobileNet():
    mobileNet =  MobileNet(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

    output    = mobileNet.layers[-1].output
    output    = keras.layers.Flatten()(output)
    ModelmobileNet = Model(inputs=mobileNet.input, outputs=output)# base_model.get_layer('custom').output)

    ModelmobileNet.trainable = False
    for layer in ModelmobileNet.layers:
        layer.trainable = False
    return ModelmobileNet

In [5]:
def getFeaturesMobileNet(model, input_imgs):
    features = model.predict(input_imgs, verbose=0)
    return features

In [7]:
print("Loading Data ... ")
for name in os.listdir(path):
    print(name)
    for file in os.listdir(path + "/" + name + "/no_mask"):
        if file == ".DS_Store":
            continue
        filename = path + "/" + name + "/no_mask/" + file
        
        
        
        img = cv2.imread(filename)
        img = cv2.resize(img, size)
        ftr_list.append(img)
        name_list.append("no_mask")
        
    for file in os.listdir(path + "/" + name + "/mask"):
        if file == ".DS_Store":
            continue
        filename = path + "/" + name + "/mask/" + file
        img = cv2.imread(filename)
        img = cv2.resize(img, size)
        ftr_list.append(img)
        name_list.append("mask")

Loading Data ... 
luluk
devi
firsa
febri
sintia
supardi
tikah
rima
hidayatul


In [8]:
name_np = np.array(name_list)
ftr_np = np.array(ftr_list)
print("Data telah terload ... Done")

Data telah terload ... Done


In [9]:
print("Check komposisi data")
(unique, counts) = np.unique(name_np, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

Check komposisi data
[['mask' '389']
 ['no_mask' '398']]


In [10]:
less5=unique[counts<5]
print("Data yg kurang dari 5")
print(less5)

Data yg kurang dari 5
[]


In [11]:
print("Hapus data yg kurang dari 5")
for i in range(0,len(less5)):
    idx = np.where(name_np==less5[i])
    ftr_np = np.delete(ftr_np,idx,axis=0)
    nrp_np = np.delete(name_np,idx)
(unique, counts) = np.unique(name_np, return_counts=True)
numClass=len(unique)

Hapus data yg kurang dari 5


In [12]:
print(numClass)

2


In [13]:
#Melabeli nama dengan angka
#ex : anisa:0, dafa:1, danang:2
mapLabel = dict()
for i in np.unique(name_np):
    mapLabel[i] = len(mapLabel)

In [14]:
print(mapLabel)

{'mask': 0, 'no_mask': 1}


In [15]:
#generate nilai 0,1,2 untuk tiap data
classLabel=[]                        
for i in name_np:
    classLabel.append(mapLabel[i])

In [16]:
print(classLabel)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [17]:
name_category = to_categorical(classLabel,numClass)

In [18]:
# Parse numbers as floats
ftr_np_float = ftr_np.astype('float32')
# Normalize data
ftr_np_float = ftr_np / 255

In [19]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

In [20]:
batch_size = 32
fold_no = 1
verbosity = 1

In [21]:
ModelmobileNet = createMobileNet()
early_stopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')

In [22]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)

In [27]:
for train_index, test_index in sss.split(ftr_np_float, name_category):
    x_train, x_test = ftr_np_float[train_index], ftr_np_float[test_index]
    y_train, y_test = name_np[train_index], name_np[test_index]
    print("mobileNet Predict for get Features")
    print("##################################")    
    x_trainMobileNet = getFeaturesMobileNet(ModelmobileNet, x_train)
    x_testMobileNet  = getFeaturesMobileNet(ModelmobileNet, x_test)
   

    print('Train Bottleneck Features:', x_trainMobileNet.shape, 
              '\nValidation Bottleneck Features:', x_testMobileNet.shape)
    
    print("Created Model %d" %fold_no)#-> model_save)
    print("#########################")    
    nmModel  = 'mobileNetNN.h5_fold_%d.h5'%fold_no#-> model_save  
    #model = LinearDiscriminantAnalysis()
    model = LogisticRegression(multi_class='auto',solver='lbfgs', max_iter=1000) 
        
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(x_trainMobileNet,y_train)    
    scores = model.score(x_testMobileNet,y_test)
    print(f'Score for fold {fold_no}: {scores}')
    acc_per_fold.append(scores * 100)
    
    # Increase fold number
    fold_no = fold_no + 1

mobileNet Predict for get Features
##################################
Train Bottleneck Features: (393, 50176) 
Validation Bottleneck Features: (394, 50176)
Created Model 3
#########################
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: 0.9873096446700508


In [28]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} -  Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')    
print('------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 -  Accuracy: 98.73096446700508%
------------------------------------------------------------------------
> Fold 2 -  Accuracy: 98.73096446700508%
------------------------------------------------------------------------
> Fold 3 -  Accuracy: 98.73096446700508%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 98.73096446700508 (+- 0.0)
------------------------
